In [22]:
import numpy as np
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier

In [44]:
all_train = []

for i in range(100):
    fname = 'results_42876/UncSampling/training_samples/initialtrain_v' + str(i) + '.csv'
    data = pd.read_csv(fname, index_col=False)
    all_train.append(data)
    
train = pd.concat(all_train, ignore_index=True)

In [45]:
mean_train = train.sample(n=70, replace=False)

In [46]:
sum(mean_train['type'] == 'Ia')

35

In [47]:
mean_train.to_csv('results_42876/UncSampling/training_samples/mean_train.csv', index=False)

In [48]:
features = pd.read_csv('results_42876/data/features_42876.csv', index_col=False)

In [49]:
mean_train.keys()

Index(['a_g', 'b_g', 'c_g', 'snratio_g', 'chisq_g', 'nrise_g', 'a_r', 'b_r',
       'c_r', 'snratio_r', 'chisq_r', 'nrise_r', 'objectId', 'type'],
      dtype='object')

In [50]:
train_flag = np.array([item in mean_train['objectId'].values for item in features['id'].values])

In [51]:
mean_test = features[~train_flag]

In [52]:
for name in mean_test['id'].values:
    if name in mean_train['objectId'].values:
        print(name)

In [53]:
mean_test.to_csv('results_42876/UncSampling/test_samples/mean_test.csv', index=False)

In [65]:
list(mean_test.keys())

['id',
 'type',
 'a_g',
 'b_g',
 'c_g',
 'snratio_g',
 'chisq_g',
 'nrise_g',
 'a_r',
 'b_r',
 'c_r',
 'snratio_r',
 'chisq_r',
 'nrise_r']

In [60]:
clf = RandomForestClassifier(random_state=42, n_estimators=1000)
clf.fit(mean_train[list(mean_train.keys())[:-2]], mean_train['type'].values == 'Ia')

RandomForestClassifier(n_estimators=1000, random_state=42)

In [66]:
clf.score(mean_test[list(mean_test.keys())[2:]], mean_test['type'].values == 'Ia')

0.7867404863690518